In [43]:
#--imports---
import os
import scanpy as sc
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
import numpy as np


In [90]:
#---load data and Neuronal Networks---
#set base path to load data: goes back one directory and then into the data
base_path = os.path.join('..', 'data')
#Name of data set
dataset_name = 'large_atac_gene_activity'
# read dataset into an anndata object:  Category - Cells of the brain
inPath = os.path.join(base_path, f"{dataset_name}.h5ad")
adata = sc.read(inPath)

#set base path to load data: goes back one directory and then into the data
base_path = os.path.join('..', 'src', 'models', 'saved_models')
#Name of autoencoder
dataset_name = 'autoencoder_mselossfunction'
# load autoencoder
inPath = os.path.join(base_path, f"{dataset_name}.keras")
autoencoder = tf.keras.models.load_model(inPath)

#Name of discriminator
dataset_name = 'discriminator_pretrained'
# load discriminator
inPath = os.path.join(base_path, f"{dataset_name}.keras")
discriminator = tf.keras.models.load_model(inPath)
print(discriminator.input_shape)

(None, 3580)


In [91]:
#---Prepare Data---
#ADATA->NUMPY
INPUT = adata.X.toarray()

"""
removed for testing
"""
#Encode the cell labels as One hot vector to use as additional information
#encoder = OneHotEncoder(sparse_output=False)  # `sparse=False` returns a dense array
#encoded_labels = encoder.fit_transform(adata.obs[['final_cell_label']])
# Concatenate gen expreesion matrix with oneHotLabels
#INPUT = np.concatenate((X_numpy, encoded_labels), axis=1)

#One-hot encoded Batches
encoder = OneHotEncoder(sparse_output=False)  # `sparse=False` returns a dense array
OUTPUT = encoder.fit_transform(adata.obs[['batchname_all']])
OUTPUT

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [46]:
test = np.concatenate((INPUT[0:30], encoded_labels[0:30]), axis=1)
results = discriminator(test)
entropy = tf.keras.losses.categorical_crossentropy(OUTPUT[0:30], results)
print(test.shape)
print(entropy)

(30, 3587)
tf.Tensor(
[1.05611710e-02 2.02902779e-03 4.36092820e-03 5.02762850e-03
 6.56732358e-03 1.02326252e-01 1.90089177e-03 2.93324934e-03
 2.05984013e-03 3.25875403e-03 6.48334622e-02 1.11935735e-02
 5.98306162e-03 2.41488498e-03 9.33957193e-03 3.80708324e-03
 5.61946201e+00 1.29462034e-01 3.99731332e-03 3.58747900e-03
 2.89640203e-03 3.89555260e-03 3.86752840e-03 5.10721328e-03
 8.01295638e-02 1.88323352e-02 9.47573222e-03 5.61399013e-03
 4.86520386e+00 1.58708449e-03], shape=(30,), dtype=float32)


In [104]:
print(autoencoder.input_shape)
print("INPUT shape:", INPUT[0].shape)
autoencoder(INPUT[0:1])

(None, 3580)
INPUT shape: (3580,)


<tf.Tensor: shape=(1, 3580), dtype=float32, numpy=array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [134]:
from numpy import average


y_pred = autoencoder(INPUT[0:30])
print(f"y_pred shape{y_pred.shape}")
expected_batch = OUTPUT[0:30]
print(f"real answer shape{expected_batch.shape}") 
discriminator_predictions = discriminator(y_pred)
print(f"discriminator_predictions shape{discriminator_predictions.shape}")

CCE = tf.keras.losses.categorical_crossentropy(expected_batch, discriminator_predictions)
print(CCE)
CCE = -tf.math.reduce_mean(CCE)
print(CCE)


y_pred shape(30, 3580)
real answer shape(30, 11)
discriminator_predictions shape(30, 11)
tf.Tensor(
[0.22242995 0.317155   1.0178852  0.08984765 0.29704508 2.4212985
 0.21578556 0.08989855 0.6468312  0.13315095 5.5451536  1.4344335
 0.36133453 0.26281253 1.6896727  0.14549033 4.065153   2.3337574
 0.22413823 0.2962553  0.12029588 1.3646559  0.35546654 0.26149848
 1.7381909  2.2296028  0.77218586 0.5642071  7.23871    0.25411874], shape=(30,), dtype=float32)
tf.Tensor(-1.2236154, shape=(), dtype=float32)


In [135]:
def fool_discriminator_loss(discriminator, expected_output):
    def lossfunction(y_true, y_pred):
        print(y_pred)
        # Extract the first 30 states for testing
        expected_batch =  discriminator(y_true)
        print(f"y_pred shape: {y_pred.shape}")
        print(f"real answer shape{expected_batch.shape}")
        # Pass autoencoder output to the discriminator
        discriminator_predictions = discriminator(y_pred)
        print(f"discriminator_predictions shape{discriminator_predictions.shape}")
        # Calculate the loss
        CCE = tf.keras.losses.categorical_crossentropy(expected_batch, discriminator_predictions)
        print(CCE)
        CCE = -tf.math.reduce_mean(CCE)
        print(CCE)
        # We want to maximize CCE (discriminator being wrong), so we minimize -CCE
        return CCE  # Much safer than 1/CCE
    
    return lossfunction

# Compile your autoencoder with the new loss
autoencoder.compile(
    optimizer=autoencoder.optimizer,
    loss=fool_discriminator_loss(discriminator, OUTPUT),
    metrics=autoencoder.metrics
)
print(INPUT.shape)
print(autoencoder.input_shape)
print(discriminator.input_shape)
# Continue training your generator - use the consistent variable name
autoencoder.fit(
    INPUT, INPUT,
    epochs=10,
    batch_size=30
)

(84813, 3580)
(None, 3580)
(None, 3580)
Epoch 1/10
Tensor("functional_2_1/sequential_1_2/dense_1_1/Relu:0", shape=(None, 3580), dtype=float32)
y_pred shape: (None, 3580)
real answer shape(None, 11)
discriminator_predictions shape(None, 11)
Tensor("compile_loss/lossfunction/softmax_cross_entropy_with_logits/Reshape_2:0", shape=(None,), dtype=float32)
Tensor("compile_loss/lossfunction/Neg:0", shape=(), dtype=float32)


TypeError: Mean.update_state() got multiple values for argument 'sample_weight'